In [1]:
import pandas as pd
import numpy as np
# import os
import nltk
import re
# nltk.data.path.append(r"C:\Users\liufa\nltk_data")

# from nltk.tokenize import word_tokenize
# nltk.download('brown')
import matplotlib.pyplot as plt
import collections
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


# BERT
import tensorflow as tf
import keras
from tensorflow.keras import backend as K

from tensorflow.keras.preprocessing.text import one_hot, Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Dense, Flatten, Lambda,\
    GlobalMaxPooling1D, LSTM, Bidirectional, concatenate, Embedding, multiply
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model


from transformers import AutoTokenizer, TFAutoModel, TFAutoModelForSequenceClassification, AdamW
from transformers import TFBertModel, TFDistilBertModel, TFBertForSequenceClassification

In [2]:
# raw_data=pd.read_csv(r'../input/commonlitreadabilityprize/train.csv')
raw_data=pd.read_csv(r'C:/Users/liufa/Desktop/Kaggle competition/train.csv')

df = raw_data.rename(columns={'excerpt':'text','target':'y'})
df.head(5)

,id,url_legal,license,text,y,standard_error
0,c12129c31,NaN,NaN,When the young people returned to the ballroom...,-0.340259,0.464009
1,85aa80a4c,NaN,NaN,"All through dinner time, Mrs. Fayre was somewh...",-0.315372,0.480805
2,b69ac6792,NaN,NaN,"As Roger had predicted, the snow departed as q...",-0.580118,0.476676
3,dd1000b26,NaN,NaN,And outside before the palace a great garden w...,-1.054013,0.450007
4,37c1b32fb,NaN,NaN,Once upon a time there were Three Bears who li...,0.247197,0.510845


In [3]:
import string
string.punctuation

def remove_punct(text):
    text_nopunct=''.join([char for char in text if char not in string.punctuation])
    return text_nopunct

df['text_no_punct']=df['text'].apply(lambda x: remove_punct(x))

def tokenize(text):
    tokens=re.split('\W+',text)
    return tokens
df['text_tokenized']=df['text_no_punct'].apply(lambda x: tokenize(x.lower()))

stopwords=nltk.corpus.stopwords.words('english')
def remove_stopwords(tokenized_list):
    text=[word for word in tokenized_list if word not in stopwords]
    return text
df['text_no_stopwords']=df['text_tokenized'].apply(lambda x: remove_stopwords(x))

wn=nltk.WordNetLemmatizer()
def lemmatizing(tokenized_text):
    text=[wn.lemmatize(word) for word in tokenized_text]
    return text
df['text_lemmatizer']=df['text_no_stopwords'].apply(lambda x: lemmatizing(x))

df['text_clean']=df['text_lemmatizer'].apply(lambda x: ' '.join(x))
df.head()


,id,url_legal,license,text,y,standard_error,text_no_punct,text_tokenized,text_no_stopwords,text_lemmatizer,text_clean
0,c12129c31,NaN,NaN,When the young people returned to the ballroom...,-0.340259,0.464009,When the young people returned to the ballroom...,"[when, the, young, people, returned, to, the, ...","[young, people, returned, ballroom, presented,...","[young, people, returned, ballroom, presented,...",young people returned ballroom presented decid...
1,85aa80a4c,NaN,NaN,"All through dinner time, Mrs. Fayre was somewh...",-0.315372,0.480805,All through dinner time Mrs Fayre was somewhat...,"[all, through, dinner, time, mrs, fayre, was, ...","[dinner, time, mrs, fayre, somewhat, silent, e...","[dinner, time, mr, fayre, somewhat, silent, ey...",dinner time mr fayre somewhat silent eye resti...
2,b69ac6792,NaN,NaN,"As Roger had predicted, the snow departed as q...",-0.580118,0.476676,As Roger had predicted the snow departed as qu...,"[as, roger, had, predicted, the, snow, departe...","[roger, predicted, snow, departed, quickly, ca...","[roger, predicted, snow, departed, quickly, ca...",roger predicted snow departed quickly came two...
3,dd1000b26,NaN,NaN,And outside before the palace a great garden w...,-1.054013,0.450007,And outside before the palace a great garden w...,"[and, outside, before, the, palace, a, great, ...","[outside, palace, great, garden, walled, round...","[outside, palace, great, garden, walled, round...",outside palace great garden walled round fille...
4,37c1b32fb,NaN,NaN,Once upon a time there were Three Bears who li...,0.247197,0.510845,Once upon a time there were Three Bears who li...,"[once, upon, a, time, there, were, three, bear...","[upon, time, three, bears, lived, together, ho...","[upon, time, three, bear, lived, together, hou...",upon time three bear lived together house wood...


In [4]:
def count_punct(text):
    ct=sum([1 for char in text if char in string.punctuation])
    return round(ct/(len(text)-text.count(' ')),3)

def count_capital_words(text):
    return sum(map(str.isupper,text.split()))
    
def count_sent(text):
    return len(nltk.sent_tokenize(text))

def count_stopwords(text):
    stopwords=nltk.corpus.stopwords.words('english')
    word_tokens = nltk.word_tokenize(text)
    n_stopwords = [w for w in word_tokens if w in stopwords]
    return len(n_stopwords)

def count_unique_words(text):
    return len(set(text.split()))

df['num_char']=df['text'].apply(lambda x: len(x)-x.count(' '))
df['num_words']=df['text_tokenized'].apply(lambda x: len(x))
df['num_sent']=df['text'].apply(lambda x: count_sent(x))
df['avg_word_len'] = df['num_char']/df['num_words']
df['avg_sent_len'] = df['num_char']/df['num_sent']
df['num_cap_words'] = df['text'].apply(lambda x: count_capital_words(x))
df['punct_ratio']=df['text'].apply(lambda x:count_punct(x))
df['stop_words_ratio'] = df['text'].apply(lambda x: count_stopwords(x))/df['num_words']
df['unique_words_ratio'] = df['text'].apply(lambda x: count_unique_words(x))/df['num_words']

df.head()

,id,url_legal,license,text,y,standard_error,text_no_punct,text_tokenized,text_no_stopwords,text_lemmatizer,text_clean,num_char,num_words,num_sent,avg_word_len,avg_sent_len,num_cap_words,punct_ratio,stop_words_ratio,unique_words_ratio
0,c12129c31,NaN,NaN,When the young people returned to the ballroom...,-0.340259,0.464009,When the young people returned to the ballroom...,"[when, the, young, people, returned, to, the, ...","[young, people, returned, ballroom, presented,...","[young, people, returned, ballroom, presented,...",young people returned ballroom presented decid...,819,179,11,4.575419,74.454545,0,0.033,0.452514,0.636872
1,85aa80a4c,NaN,NaN,"All through dinner time, Mrs. Fayre was somewh...",-0.315372,0.480805,All through dinner time Mrs Fayre was somewhat...,"[all, through, dinner, time, mrs, fayre, was, ...","[dinner, time, mrs, fayre, somewhat, silent, e...","[dinner, time, mr, fayre, somewhat, silent, ey...",dinner time mr fayre somewhat silent eye resti...,774,169,15,4.579882,51.600000,6,0.072,0.396450,0.751479
2,b69ac6792,NaN,NaN,"As Roger had predicted, the snow departed as q...",-0.580118,0.476676,As Roger had predicted the snow departed as qu...,"[as, roger, had, predicted, the, snow, departe...","[roger, predicted, snow, departed, quickly, ca...","[roger, predicted, snow, departed, quickly, ca...",roger predicted snow departed quickly came two...,747,166,11,4.500000,67.909091,2,0.063,0.451807,0.771084
3,dd1000b26,NaN,NaN,And outside before the palace a great garden w...,-1.054013,0.450007,And outside before the palace a great garden w...,"[and, outside, before, the, palace, a, great, ...","[outside, palace, great, garden, walled, round...","[outside, palace, great, garden, walled, round...",outside palace great garden walled round fille...,747,164,5,4.554878,149.400000,0,0.044,0.414634,0.719512
4,37c1b32fb,NaN,NaN,Once upon a time there were Three Bears who li...,0.247197,0.510845,Once upon a time there were Three Bears who li...,"[once, upon, a, time, there, were, three, bear...","[upon, time, three, bears, lived, together, ho...","[upon, time, three, bear, lived, together, hou...",upon time three bear lived together house wood...,577,147,5,3.925170,115.400000,0,0.055,0.496599,0.346939


In [5]:
# Topic modeling

In [6]:
# Train test split
cols = [c for c in df.columns if c not in ['id',
                                           'url_legal',
                                           'license',
                                           'standard_error',
                                           'text_clean',
                                           'text_no_punct',
                                           'text_tokenized',
                                           'text_no_stopwords',
                                           'text_lemmatizer',
                                           'y']]
X = df[cols]
y = df[['y']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_num = X_train[[c for c in X.columns if c != 'text']].to_numpy()
X_test_num = X_test[[c for c in X.columns if c != 'text']].to_numpy()

y_train = y_train.values
y_test = y_test.values


# standard scaler
scaler = StandardScaler().fit(X_train_num)
X_train_num = scaler.transform(X_train_num)
X_test_num = scaler.transform(X_test_num)

X_train_num

array([[ 0.98142114,  0.22000603, -0.76107608, ..., -0.66349325,
        -0.55668465,  1.36510705],
       [-1.38668274, -1.49922092, -0.76107608, ..., -0.97586316,
        -0.41923363,  1.53478439],
       [-1.16527466, -1.32136986, -0.31725371, ..., -0.03875343,
        -0.21277577,  1.24941573],
       ...,
       [ 1.37610512,  0.99069397,  0.12656867, ..., -0.35112334,
        -0.40765153,  0.6990866 ],
       [ 0.33644976, -0.37283085, -0.98298727, ...,  0.42980143,
        -0.21205825,  1.05782364],
       [-1.31929767, -1.20280248,  4.34288121, ...,  6.2867372 ,
        -1.97179774, -5.12382892]])

In [7]:
# BERT tokenizer
# 'bert-base-cased', 'distilbert-base-cased', 'roberta-base', 'distilroberta-base'
model_path = 'roberta-base'
max_len = 300

tokenizer = AutoTokenizer.from_pretrained(model_path)
def tokenize_data(X):
    tokenized = tokenizer(X.tolist(), padding='max_length',truncation=True, max_length=max_len, return_tensors="np") 
    return {feat: tokenized[feat] for feat in tokenizer.model_input_names}
    
X_train_tokenized = tokenize_data(X_train['text'])
X_test_tokenized = tokenize_data(X_test['text'])

print (
str(len(X_train_tokenized))\
+ ' x ' +\
str(len(X_train_tokenized['input_ids']))\
+ ' x ' +\
str(len(X_train_tokenized['input_ids'][0]))
)

X_train_tokenized

2 x 2267 x 300


{'input_ids': array([[    0,   133,   745, ...,     1,     1,     1],
        [    0,   133,  1114, ...,     1,     1,     1],
        [    0, 37818,     5, ...,     1,     1,     1],
        ...,
        [    0,   100,  1017, ...,     1,     1,     1],
        [    0,  1121,   209, ...,     1,     1,     1],
        [    0, 39488,    16, ...,     1,     1,     1]]),
 'attention_mask': array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]])}

In [8]:
y_train

array([[-1.51834981],
       [-0.54880659],
       [-0.19326203],
       ...,
       [-1.58438395],
       [-2.03468808],
       [-0.6052157 ]])

In [9]:
# help functions

# to define 'rmse' as loss instead of 'mse'
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true))) 

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', 
    factor = 0.5 , 
    patience=1, 
    mode='min', 
    verbose=1)

early_stop = keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=3,
    verbose=1,
    restore_best_weights=True)

In [10]:
# # Base BERT embedding + TF-IDF + numerical features

# BERTmodel = TFBertModel.from_pretrained(model_path)

# train_sent_embedding = []
# for x in a:
#     train_sent_embedding.append(BERTmodel(x).last_hidden_state[0][0])
    

# df_final = train_final.join(train_sent_embedding)
# df_final.head()

# # MLP

In [11]:
# # Ensemble model architecture BERT classifier + numerical features
def softMaxAxis1(x):
    return tf.keras.activations.softmax(x,axis=1)

input_ids = tf.keras.Input(shape=(max_len, ),dtype='int32')
attention_mask = tf.keras.Input(shape=(max_len, ),dtype='int32')

feat_num = X_train_num.shape[1]
numeric_input = tf.keras.Input(shape=(feat_num,),dtype='float32')

# change huggingface model if needed
bert_layer = TFAutoModel.from_pretrained(model_path)(input_ids, attention_mask)[0]

# cls_token = bert_layer[:, 0, :]

# attention layer
weight_layer = Dense(512, activation='tanh')(bert_layer)
weight_layer = Dense(1, activation=softMaxAxis1)(weight_layer)
layer = multiply([weight_layer,bert_layer])
layer = Lambda(lambda x: K.sum(x, axis=1))(layer)
layer = Dense(270, activation='relu')(layer)

num_layer = Dense(9, activation='relu')(numeric_input)
num_layer = Dropout(0.3)(num_layer)

layer = concatenate([layer, num_layer])
# layer = Dense(180, activation='relu')(layer)
# layer = Dropout(0.2)(layer)
out = Dense(1, activation='linear')(layer)

model = tf.keras.Model(inputs=[input_ids , attention_mask, numeric_input], outputs=out)

# freezing BERT embeddings
# model.layers[2].trainable = False

model.compile(optimizer=Adam(2e-5), loss=rmse) # small training rates are necessary!

model.summary()

Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
Model: "model"
__________________________________________________________________________________________________
Layer 

In [12]:
history = model.fit([X_train_tokenized['input_ids'],\
                     X_train_tokenized['attention_mask'],\
                     X_train_num],\
                    y_train, epochs=15, verbose=1, batch_size=3, validation_split=0.35, callbacks=[reduce_lr,early_stop])

score = model.evaluate([X_test_tokenized['input_ids'],\
                        X_test_tokenized['attention_mask'],\
                        X_test_num],\
                       y_test, verbose=1)
print("Test Scores:", score)

K.clear_session()

Epoch 1/15
491/491 [==============================] - 114s 219ms/step - loss: 0.6607 - val_loss: 0.5829
Epoch 2/15
491/491 [==============================] - 106s 215ms/step - loss: 0.5202 - val_loss: 0.5700
Epoch 3/15
491/491 [==============================] - 108s 220ms/step - loss: 0.4157 - val_loss: 0.5104
Epoch 4/15
491/491 [==============================] - 110s 223ms/step - loss: 0.3528 - val_loss: 0.5457

Epoch 00004: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
Epoch 5/15
491/491 [==============================] - 104s 211ms/step - loss: 0.2731 - val_loss: 0.5564

Epoch 00005: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-06.
Epoch 6/15
491/491 [==============================] - 104s 211ms/step - loss: 0.2110 - val_loss: 0.5220

Epoch 00006: ReduceLROnPlateau reducing learning rate to 2.499999936844688e-06.
Restoring model weights from the end of the best epoch.
Epoch 00006: early stopping
18/18 [==============================] - 7s 386ms

In [28]:
# model.layers[2].output
K.clear_session()

In [ ]:
from kerastuner.tuners import BayesianOptimization
def build_model(hp):
    input_ids = tf.keras.Input(shape=(max_len, ),dtype='int32')
    attention_mask = tf.keras.Input(shape=(max_len, ),dtype='int32')

    feat_num = X_train_num.shape[1]
    numeric_input = tf.keras.Input(shape=(feat_num,),dtype='float32')

    # change huggingface model if needed
    bert_layer = TFAutoModel.from_pretrained(model_path)(input_ids, attention_mask)[0]

    layer = Flatten()(bert_layer)
    layer = Dense(769, activation='relu')(layer)
    layer = Concatenate(axis=1)([layer, numeric_input])
    layer = Dense(hp.Int('dense_units1',min_value=40,max_value=300,step=10), activation='relu')(layer)
    layer = Dropout(0.2)(layer)
    out = Dense(1, activation='linear')(layer)

    model = tf.keras.Model(inputs=[input_ids , attention_mask, numeric_input], outputs=out)

#     model.compile(optimizer=Adam(2e-5), loss='mse', metrics=['mean_squared_error']) 
    model.compile(optimizer=Adam(2e-5), loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError()]) 
    return model

In [ ]:
import os

bayesian_opt_tuner = BayesianOptimization(
    build_model,
    objective='val_mean_squared_error',
    max_trials=10,
    executions_per_trial=1,
    directory=os.path.normpath(r'C:\Users\liufa\keras_tuning'),
    project_name='kerastuner_bayesian_poc',
    overwrite=True)

bayesian_opt_tuner.search([X_train_tokenized['input_ids'],X_train_tokenized['attention_mask'],X_train_num],\
                           y_train, epochs=15, batch_size=2, callbacks=[early_stop], validation_split=0.3, verbose=1)
     #validation_data=(X_test, y_test)
#     callbacks=[early_stop], 

bayes_opt_model_best_model = bayesian_opt_tuner.get_best_models(num_models=2)

In [ ]:
bayes_opt_model_best_model[0].summary()
best_hp = bayesian_opt_tuner.get_best_hyperparameters(1)[0]
print(best_hp.values)

In [ ]:
################################ Submission ##########################################

df_test = pd.read_csv(r'../input/commonlitreadabilityprize/test.csv')

df_test = df_test.rename(columns={'excerpt':'text','target':'y'})
df_test['text_no_punct']=df_test['text'].apply(lambda x: remove_punct(x))
df_test['text_tokenized']=df_test['text_no_punct'].apply(lambda x: tokenize(x.lower()))
stopwords=nltk.corpus.stopwords.words('english')
df_test['text_no_stopwords']=df_test['text_tokenized'].apply(lambda x: remove_stopwords(x))
wn=nltk.WordNetLemmatizer()
df_test['text_lemmatizer']=df_test['text_no_stopwords'].apply(lambda x: lemmatizing(x))
df_test['text_clean']=df_test['text_lemmatizer'].apply(lambda x: ' '.join(x))
df_test['sentence_len']=df_test['text'].apply(lambda x: len(x)-x.count(' '))
df_test['word_len']=df_test['text_tokenized'].apply(lambda x: len(x))
df_test['punct_count']=df_test['text'].apply(lambda x:count_punct(x))
df_test['d2v_vector'] = df_test['text_lemmatizer'].apply(lambda x:d2v_model.infer_vector(x))

X_test_LSTM = df_test['text_clean']
X_test_LSTM = tokenizer.texts_to_sequences(X_test_LSTM)
X_test_LSTM = pad_sequences(X_test_LSTM, padding='post', maxlen=maxlen)
df_test['lstm_feature0'] = model.predict(X_test_LSTM)


test_d2v_features = pd.DataFrame(df_test["d2v_vector"].to_list(), columns=['d2v_feature'+str(i) for i in range(40)])
X_test_ridge = df_test.drop(columns = ['id','url_legal','license','text','y','standard_error',
 'text_no_punct','text_tokenized','text_no_stopwords','text_lemmatizer','text_clean','d2v_vector'
                        ])

test_final = df_test.join(test_d2v_features)


In [ ]:
df_test['target'] = ridge.predict(X_test_ridge)
output = df_test[['id','target']]
output.to_csv('./submission.csv', index=False)